In [1]:
# mounting drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/BullingerDigitalLMFootnotes/git_repo


/content/drive/MyDrive/NLP_2_Conspiracies/code/task/LM_classification_huggingface


## Run LLama3 continuing the TEI

In [1]:
%pip install accelerate

In [2]:
%pip install bitsandbytes

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [116]:

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            text = tokenizer.decode(input_ids[0])
            # Todo: find a nice way to watch the generation
            # print(text.split()[-1])
            # need to use the text, as the same text can be tokenized differently depending on context...
            # no guarantee that it ends with the text, as end token may be ">." or something...
            if stop in text[-10:]:
                return True
        return False

stop_words = ["</note>"]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words)])


def generate(prompt:str, model, tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt")['input_ids'].to(model.device)
    # terminators = tokenizer.encode("</note>", add_special_tokens=False)
    # print(terminators)
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            do_sample=True,
            # eos_token_id=terminators,
            temperature=0.7,
            max_new_tokens=500,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=stopping_criteria  # stop when "</note>" is generated
            )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [4]:
import transformers
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Quantization, as shown here: https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing#scrollTo=VPD7QS_DR-mw
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [19]:
text = """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file10224" type="Brief" source="HBBW-3" n="193">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Konrad Geßner, Johannes Fries / Basel an Heinrich Bullinger, 25. Februar [1533]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Optimo et integerrimo viro M. Henrico Bullingero, mecaenati charissimo.</s>
				</p>
				<p>
					<s n="2" xml:lang="la" type="auto">S.</s>
					<s n="3" xml:lang="la" type="auto">Impediunt nos ab itinere<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> befanden sich auf dem Weg nach <placeName ref="l59" cert="high">Bourges</placeName>.</note> nives, pluvia et ventorum vis.</s>
					<s n="4" xml:lang="la" type="auto"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName><note xml:id="fn4" type="footnote" n="4">"""

In [120]:
for i in range(10):
  print(generate(text, model, tokenizer))


<persName ref="p8418" cert="high">Myconius</persName> war <persName ref="p1222" cert="high">Fries</persName>' Freund in <placeName ref="l59" cert="high">Bourges</placeName> und als <date>1529</date> von dort nach <placeName ref="l28" cert="high">Basel</placeName> berufen worden.</note></
Zu <persName ref="p8418" cert="high">Myconium</persName> s. <ref target="p8418">n. 19</ref>. <persName ref="p8418" cert="high">Myconium</persName> war ein Basler Ratsherr (1499–1525). <persName ref="p8418" cert="high">Myconium</persName> hatte in <placeName ref="l41" cert="low">Straßburg</placeName> den <persName ref="p1011" cert="high">Schwenckfelder</persName> <persName ref="p1056" cert="high">Pirkheimer</persName> und <persName ref="p1061" cert="high">Schmid</persName> kennen gelernt. Die <persName ref="p8418" cert="high">Myconium</persName> waren <dateRange key="date1" cert="high" type="year">1525</dateRange>–<dateRange key="date2" cert="high" type="year">1541</dateRange> die Gastgeber der <placeNa

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!python COT_instruct.py meta-llama/Meta-Llama-3-8B-Instruct ../../data/dataset_en_split_dev.json instruct_prompt2.jsonl --few_shot examples.jsonl

new_
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100% 4/4 [01:20<00:00, 20.14s/it]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
download done
  0% 0/400 [00:00<?, ?it/s]2024-05-16 15:40:06.476520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [ ]:
import torch
torch.cuda.empty_cache()